In [1]:
import tensorflow as tf 

In [2]:
print(tf.__version__)

1.15.2


In [11]:
from mrcnn import model as modellib, config, utils
from mrcnn.visualize import display_instances
import os
import numpy as np 
import matplotlib.pyplot as plt 
import cv2
import datetime
import skimage.draw
import json
import Paths


In [ ]:
Path=Paths.ProjectPath()

In [12]:
ROOT_DIR=Path.ROOT_DIR
DSPath=Path.DatasetPath
LogPath=Path.MRCNN_Training_Logs_Path
coco_wgt_path="E://vsc2.0//GitHub//Grape_instance_segementation_and_masking//mask_rcnn_coco.h5"

In [13]:
class CustomConfig(config.Config):
    NAME="object"
    NUM_CLASSES = 1+1
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1
    STEPS_PER_EPOCH=150
    DETECTION_MIN_CONFIDENCE=0.5
    LEARNING_RATE=0.0005

In [14]:
class CustomDataset(utils.Dataset):
    def load_custom(self,DSPath,subset):
        self.add_class("object",1,"Grapes")
        # self.add_class("object",2,"CFR Grapes")
        # self.add_class("object",3,"CSV Grapes")
        # self.add_class("object",4,"SVB Grapes")
        # self.add_class("object",5,"SYH Grapes")
        
        data=json.load(open(os.path.join(DSPath,"MRCNN_DS.json")))
        assert subset in ['train','test']
        dataset=data[subset]
        ImgPath=os.path.join(DSPath,"Images")
        NPZPath=os.path.join(DSPath,"NPZFiles")
        TextPath=os.path.join(DSPath,"TextFiles")
        for file in dataset:
            imgfile=file[:-3]+"jpg"
            imgpath=os.path.join(ImgPath,imgfile)
            npzpath=os.path.join(NPZPath,file)
            textpath=os.path.join(TextPath,file[:-3]+"txt")
            img=cv2.imread(imgpath)
            height,width=img.shape[:2]
            self.add_image(source="object",image_id=imgfile,path=imgpath, height=height,width=width,npzfile=npzpath,textfile=textpath,filename=file)

    def yolo_to_unnormalized_box(box_line, image_width, image_height):
        yolo_box=list(map(float,box_line.split(" ")))
        boxClass, center_x, center_y, width, height = yolo_box
 
        # Convert normalized coordinates to original image coordinates
        x_center = int(center_x * image_width)
        y_center = int(center_y * image_height)
        box_width = int(width * image_width)
        box_height = int(height * image_height)

        # Calculate xmin, xmax, ymin, ymax
        x_min = x_center - box_width // 2
        y_min = y_center - box_height // 2
        x_max = x_min + box_width
        y_max = y_min + box_height

        return x_min, x_max, y_min, y_max
    
    def load_mask(self, image_id):
        image_info = self.image_info[image_id]
        if image_info["source"] != "object":
            return super(self.__class__, self).load_mask(image_id)
        
        # classes={"CDY":1,"CFR":2,"CSV":3,"SVB":4,"SYH":5}
        # filename=image_info['filename']
        # class_num=classes[filename[:3]]
        npzfile=np.load(image_info['npzfile'])
        
        masks=npzfile['arr_0']
        # masks=masks.astype(np.uint8)
        num_masks=masks.shape
        num_masks=num_masks[-1]
        class_ids=np.ones(num_masks,dtype=np.int32)
        # class_ids=class_ids*class_num
        return masks,class_ids
    
    def image_reference(self, image_id):
        """Return the path of the image."""
        info = self.image_info[image_id]
        if info["source"] == "object":
            return info["path"]
        else:
            super(self.__class__, self).image_reference(image_id)
    

In [15]:
def train(model,epochs_):
    """Train the model."""
    dataset_train = CustomDataset()
    dataset_train.load_custom(DSPath, "train")
    dataset_train.prepare()

    dataset_val = CustomDataset()
    dataset_val.load_custom(DSPath, "test")
    dataset_val.prepare()

    
    print("Training network all layers")
    model.train(dataset_train, dataset_val,
                learning_rate=config.LEARNING_RATE,
                epochs=epochs_,
                layers='all')

In [16]:
config = CustomConfig()
model = modellib.MaskRCNN(mode="training", config=config,
                                  model_dir=LogPath)

In [17]:
model.load_weights(coco_wgt_path, by_name=True)
"

' exclude=[\n            "mrcnn_class_logits", "mrcnn_bbox_fc",\n            "mrcnn_bbox", "mrcnn_mask"])'

In [10]:
train(model,20)

Training network all layers

Starting at epoch 0. LR=0.0005

Checkpoint Path: E://vsc2.0//GitHub//Grape_instance_segementation_and_masking\logs\object20231230T0013\mask_rcnn_object_{epoch:04d}.h5
Selecting layers to train
conv1                  (Conv2D)
bn_conv1               (BatchNorm)
res2a_branch2a         (Conv2D)
bn2a_branch2a          (BatchNorm)
res2a_branch2b         (Conv2D)
bn2a_branch2b          (BatchNorm)
res2a_branch2c         (Conv2D)
res2a_branch1          (Conv2D)
bn2a_branch2c          (BatchNorm)
bn2a_branch1           (BatchNorm)
res2b_branch2a         (Conv2D)
bn2b_branch2a          (BatchNorm)
res2b_branch2b         (Conv2D)
bn2b_branch2b          (BatchNorm)
res2b_branch2c         (Conv2D)
bn2b_branch2c          (BatchNorm)
res2c_branch2a         (Conv2D)
bn2c_branch2a          (BatchNorm)
res2c_branch2b         (Conv2D)
bn2c_branch2b          (BatchNorm)
res2c_branch2c         (Conv2D)
bn2c_branch2c          (BatchNorm)
res3a_branch2a         (Conv2D)
bn3a_bran

e:\vsc2.0\GitHub\.venv\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
e:\vsc2.0\GitHub\.venv\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
e:\vsc2.0\GitHub\.venv\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "






Epoch 1/20
150/150 [==============================] - 3977s 27s/step - loss: 1.7407 - rpn_class_loss: 0.1009 - rpn_bbox_loss: 0.3081 - mrcnn_class_loss: 0.4017 - mrcnn_bbox_loss: 0.4464 - mrcnn_mask_loss: 0.4836 - val_loss: 1.2598 - val_rpn_class_loss: 0.0540 - val_rpn_bbox_loss: 0.2332 - val_mrcnn_class_loss: 0.3250 - val_mrcnn_bbox_loss: 0.3375 - val_mrcnn_mask_loss: 0.3101

Epoch 2/20
150/150 [==============================] - 3868s 26s/step - loss: 1.0319 - rpn_class_loss: 0.0409 - rpn_bbox_loss: 0.1823 - mrcnn_class_loss: 0.2715 - mrcnn_bbox_loss: 0.2525 - mrcnn_mask_loss: 0.2846 - val_loss: 1.2453 - val_rpn_class_loss: 0.0484 - val_rpn_bbox_loss: 0.2542 - val_mrcnn_class_loss: 0.3142 - val_mrcnn_bbox_loss: 0.3215 - val_mrcnn_mask_loss: 0.3070
Epoch 3/20
150/150 [==============================] - 3874s 26s/step - loss: 0.7787 - rpn_class_loss: 0.0207 - rpn_bbox_loss: 0.1341 - mrcnn_class_loss: 0.1980 - mrcnn_bbox_loss: 0.1713 - mrcnn_mask_loss: 0.2545 - val_loss: 1.2433 - val_

In [10]:
train(model,epochs_=5)

Training network all layers

Starting at epoch 0. LR=0.0005

Checkpoint Path: E://vsc2.0//GitHub//Grape_instance_segementation_and_masking\logs\object20231231T0007\mask_rcnn_object_{epoch:04d}.h5
Selecting layers to train
conv1                  (Conv2D)
bn_conv1               (BatchNorm)
res2a_branch2a         (Conv2D)
bn2a_branch2a          (BatchNorm)
res2a_branch2b         (Conv2D)
bn2a_branch2b          (BatchNorm)
res2a_branch2c         (Conv2D)
res2a_branch1          (Conv2D)
bn2a_branch2c          (BatchNorm)
bn2a_branch1           (BatchNorm)
res2b_branch2a         (Conv2D)
bn2b_branch2a          (BatchNorm)
res2b_branch2b         (Conv2D)
bn2b_branch2b          (BatchNorm)
res2b_branch2c         (Conv2D)
bn2b_branch2c          (BatchNorm)
res2c_branch2a         (Conv2D)
bn2c_branch2a          (BatchNorm)
res2c_branch2b         (Conv2D)
bn2c_branch2b          (BatchNorm)
res2c_branch2c         (Conv2D)
bn2c_branch2c          (BatchNorm)
res3a_branch2a         (Conv2D)
bn3a_bran

e:\vsc2.0\GitHub\.venv\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
e:\vsc2.0\GitHub\.venv\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
e:\vsc2.0\GitHub\.venv\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "






Epoch 1/5
150/150 [==============================] - 3905s 26s/step - loss: 0.2338 - rpn_class_loss: 9.6904e-04 - rpn_bbox_loss: 0.0162 - mrcnn_class_loss: 0.0313 - mrcnn_bbox_loss: 0.0176 - mrcnn_mask_loss: 0.1677 - val_loss: 1.7840 - val_rpn_class_loss: 0.2003 - val_rpn_bbox_loss: 0.4266 - val_mrcnn_class_loss: 0.5295 - val_mrcnn_bbox_loss: 0.2943 - val_mrcnn_mask_loss: 0.3333

Epoch 2/5
150/150 [==============================] - 3918s 26s/step - loss: 0.2360 - rpn_class_loss: 8.5058e-04 - rpn_bbox_loss: 0.0174 - mrcnn_class_loss: 0.0314 - mrcnn_bbox_loss: 0.0187 - mrcnn_mask_loss: 0.1677 - val_loss: 1.8903 - val_rpn_class_loss: 0.1601 - val_rpn_bbox_loss: 0.3904 - val_mrcnn_class_loss: 0.6876 - val_mrcnn_bbox_loss: 0.3069 - val_mrcnn_mask_loss: 0.3454
Epoch 3/5
150/150 [==============================] - 3889s 26s/step - loss: 0.2286 - rpn_class_loss: 9.0471e-04 - rpn_bbox_loss: 0.0164 - mrcnn_class_loss: 0.0311 - mrcnn_bbox_loss: 0.0159 - mrcnn_mask_loss: 0.1642 - val_loss: 1.69

In [10]:
train(model,10)

Training network all layers

Starting at epoch 0. LR=0.0005

Checkpoint Path: E://vsc2.0//GitHub//Grape_instance_segementation_and_masking\logs\object20231231T1328\mask_rcnn_object_{epoch:04d}.h5
Selecting layers to train
conv1                  (Conv2D)
bn_conv1               (BatchNorm)
res2a_branch2a         (Conv2D)
bn2a_branch2a          (BatchNorm)
res2a_branch2b         (Conv2D)
bn2a_branch2b          (BatchNorm)
res2a_branch2c         (Conv2D)
res2a_branch1          (Conv2D)
bn2a_branch2c          (BatchNorm)
bn2a_branch1           (BatchNorm)
res2b_branch2a         (Conv2D)
bn2b_branch2a          (BatchNorm)
res2b_branch2b         (Conv2D)
bn2b_branch2b          (BatchNorm)
res2b_branch2c         (Conv2D)
bn2b_branch2c          (BatchNorm)
res2c_branch2a         (Conv2D)
bn2c_branch2a          (BatchNorm)
res2c_branch2b         (Conv2D)
bn2c_branch2b          (BatchNorm)
res2c_branch2c         (Conv2D)
bn2c_branch2c          (BatchNorm)
res3a_branch2a         (Conv2D)
bn3a_bran

e:\vsc2.0\GitHub\.venv\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
e:\vsc2.0\GitHub\.venv\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
e:\vsc2.0\GitHub\.venv\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "






Epoch 1/10
150/150 [==============================] - 3968s 26s/step - loss: 0.4249 - rpn_class_loss: 0.0268 - rpn_bbox_loss: 0.0561 - mrcnn_class_loss: 0.1103 - mrcnn_bbox_loss: 0.0478 - mrcnn_mask_loss: 0.1839 - val_loss: 1.1741 - val_rpn_class_loss: 0.0819 - val_rpn_bbox_loss: 0.2835 - val_mrcnn_class_loss: 0.3533 - val_mrcnn_bbox_loss: 0.1890 - val_mrcnn_mask_loss: 0.2664

Epoch 2/10
150/150 [==============================] - 3875s 26s/step - loss: 0.3471 - rpn_class_loss: 0.0173 - rpn_bbox_loss: 0.0470 - mrcnn_class_loss: 0.0710 - mrcnn_bbox_loss: 0.0374 - mrcnn_mask_loss: 0.1744 - val_loss: 1.1286 - val_rpn_class_loss: 0.0830 - val_rpn_bbox_loss: 0.2574 - val_mrcnn_class_loss: 0.3312 - val_mrcnn_bbox_loss: 0.1975 - val_mrcnn_mask_loss: 0.2595
Epoch 3/10
150/150 [==============================] - 3872s 26s/step - loss: 0.2922 - rpn_class_loss: 0.0069 - rpn_bbox_loss: 0.0290 - mrcnn_class_loss: 0.0553 - mrcnn_bbox_loss: 0.0305 - mrcnn_mask_loss: 0.1705 - val_loss: 1.0611 - val_

In [10]:
train(model,5)

Training network all layers

Starting at epoch 0. LR=0.0005

Checkpoint Path: E://vsc2.0//GitHub//Grape_instance_segementation_and_masking\logs\object20240101T0050\mask_rcnn_object_{epoch:04d}.h5
Selecting layers to train
conv1                  (Conv2D)
bn_conv1               (BatchNorm)
res2a_branch2a         (Conv2D)
bn2a_branch2a          (BatchNorm)
res2a_branch2b         (Conv2D)
bn2a_branch2b          (BatchNorm)
res2a_branch2c         (Conv2D)
res2a_branch1          (Conv2D)
bn2a_branch2c          (BatchNorm)
bn2a_branch1           (BatchNorm)
res2b_branch2a         (Conv2D)
bn2b_branch2a          (BatchNorm)
res2b_branch2b         (Conv2D)
bn2b_branch2b          (BatchNorm)
res2b_branch2c         (Conv2D)
bn2b_branch2c          (BatchNorm)
res2c_branch2a         (Conv2D)
bn2c_branch2a          (BatchNorm)
res2c_branch2b         (Conv2D)
bn2c_branch2b          (BatchNorm)
res2c_branch2c         (Conv2D)
bn2c_branch2c          (BatchNorm)
res3a_branch2a         (Conv2D)
bn3a_bran

e:\vsc2.0\GitHub\.venv\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
e:\vsc2.0\GitHub\.venv\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
e:\vsc2.0\GitHub\.venv\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "






Epoch 1/5
150/150 [==============================] - 3844s 26s/step - loss: 0.2071 - rpn_class_loss: 8.5639e-04 - rpn_bbox_loss: 0.0128 - mrcnn_class_loss: 0.0290 - mrcnn_bbox_loss: 0.0139 - mrcnn_mask_loss: 0.1505 - val_loss: 1.2221 - val_rpn_class_loss: 0.0852 - val_rpn_bbox_loss: 0.2688 - val_mrcnn_class_loss: 0.3712 - val_mrcnn_bbox_loss: 0.2073 - val_mrcnn_mask_loss: 0.2896

Epoch 2/5
150/150 [==============================] - 3908s 26s/step - loss: 0.2021 - rpn_class_loss: 7.6079e-04 - rpn_bbox_loss: 0.0132 - mrcnn_class_loss: 0.0279 - mrcnn_bbox_loss: 0.0136 - mrcnn_mask_loss: 0.1466 - val_loss: 1.1089 - val_rpn_class_loss: 0.0778 - val_rpn_bbox_loss: 0.2215 - val_mrcnn_class_loss: 0.3500 - val_mrcnn_bbox_loss: 0.1874 - val_mrcnn_mask_loss: 0.2722
Epoch 3/5
150/150 [==============================] - 3896s 26s/step - loss: 0.1968 - rpn_class_loss: 4.9487e-04 - rpn_bbox_loss: 0.0123 - mrcnn_class_loss: 0.0268 - mrcnn_bbox_loss: 0.0130 - mrcnn_mask_loss: 0.1442 - val_loss: 1.36

In [18]:
train(model,10)

Training network all layers

Starting at epoch 0. LR=0.0005

Checkpoint Path: E://vsc2.0//GitHub//Grape_instance_segementation_and_masking\logs\object20240101T1029\mask_rcnn_object_{epoch:04d}.h5
Selecting layers to train
conv1                  (Conv2D)
bn_conv1               (BatchNorm)
res2a_branch2a         (Conv2D)
bn2a_branch2a          (BatchNorm)
res2a_branch2b         (Conv2D)
bn2a_branch2b          (BatchNorm)
res2a_branch2c         (Conv2D)
res2a_branch1          (Conv2D)
bn2a_branch2c          (BatchNorm)
bn2a_branch1           (BatchNorm)
res2b_branch2a         (Conv2D)
bn2b_branch2a          (BatchNorm)
res2b_branch2b         (Conv2D)
bn2b_branch2b          (BatchNorm)
res2b_branch2c         (Conv2D)
bn2b_branch2c          (BatchNorm)
res2c_branch2a         (Conv2D)
bn2c_branch2a          (BatchNorm)
res2c_branch2b         (Conv2D)
bn2c_branch2b          (BatchNorm)
res2c_branch2c         (Conv2D)
bn2c_branch2c          (BatchNorm)
res3a_branch2a         (Conv2D)
bn3a_bran

e:\vsc2.0\GitHub\.venv\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
e:\vsc2.0\GitHub\.venv\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
e:\vsc2.0\GitHub\.venv\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/10
150/150 [==============================] - 5127s 34s/step - loss: 0.2066 - rpn_class_loss: 0.0028 - rpn_bbox_loss: 0.0146 - mrcnn_class_loss: 0.0318 - mrcnn_bbox_loss: 0.0144 - mrcnn_mask_loss: 0.1431 - val_loss: 1.3665 - val_rpn_class_loss: 0.1274 - val_rpn_bbox_loss: 0.3669 - val_mrcnn_class_loss: 0.3704 - val_mrcnn_bbox_loss: 0.2091 - val_mrcnn_mask_loss: 0.2926
Epoch 2/10
150/150 [==============================] - 5545s 37s/step - loss: 0.1921 - rpn_class_loss: 7.3880e-04 - rpn_bbox_loss: 0.0115 - mrcnn_class_loss: 0.0281 - mrcnn_bbox_loss: 0.0116 - mrcnn_mask_loss: 0.1402 - val_loss: 1.0795 - val_rpn_class_loss: 0.0643 - val_rpn_bbox_loss: 0.2110 - val_mrcnn_class_loss: 0.3400 - val_mrcnn_bbox_loss: 0.1814 - val_mrcnn_mask_loss: 0.2829
Epoch 3/10
150/150 [==============================] - 5538s 37s/step - loss: 0.1916 - rpn_class_loss: 6.5217e-04 - rpn_bbox_loss: 0.0105 - mrcnn_class_loss: 0.0285 - mrcnn_bbox_loss: 0.0120 - mrcnn_mask_loss: 0.1400 - val_loss: 1.0094 - v